# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# have to pip install gmaps otherwise you get an error xD

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import vacation_api

# access gmaps with my API key
gmaps.configure(api_key=vacation_api)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# read the weatherpy.csv to create dataframe
cities = pd.read_csv("../Output/weatherpy_output.csv", encoding="utf-8")
cities.head()


,city_id,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,0,San-Pédro,100,CI,1667515067,89,4.7485,-6.6363,77.81,4.72
1,1,Spearfish,100,US,1667515067,53,44.4908,-103.8594,35.10,12.66
2,2,Namatanai,37,PG,1667515068,67,-3.6667,152.4333,85.28,1.23
3,3,Puerto Ayora,80,EC,1667515068,78,-0.7393,-90.3518,75.16,12.33
4,4,Punta Arenas,40,CL,1667515069,63,-53.1500,-70.9167,62.71,23.02


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# setting variables
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Long"]]

In [18]:
# setting the heat layer
# also had to enable the extension for our session "jupyter nbextension enable --py gmaps"


# customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# setting up details of the layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)

# adding the heat layer and printing the map
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# create a new DF which filters temp btw 70 & 80 degrees f, 
# wind speed less than 10 mph and 0 cloudiness 
narrowed_city_df = cities.loc[
    (cities["Max Temp"] > 70) & 
    (cities["Max Temp"] < 80) & 
    (cities["Cloudiness"] == 0), :]

# drop the rows that don't meet conditions above
narrowed_city_df = narrowed_city_df.dropna(how='any')

# reset the index to clean the DF
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']

# print to show headers
narrowed_city_df.head()

,city_id,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,16,Salta,0,AR,1667515073,16,-24.7859,-65.4117,73.99,12.66
1,55,Sur,0,OM,1667515087,49,22.5667,59.5289,77.25,10.07
2,164,Rawson,0,AR,1667514984,25,-43.3002,-65.1023,73.00,24.81
3,176,Resistencia,0,AR,1667515072,32,-27.4606,-58.9839,76.98,5.75
4,201,Mitsamiouli,0,KM,1667515141,78,-11.3847,43.2844,76.17,9.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    long = narrowed_city_df.loc[i]['Long']

    params = {
        "location": f"{lat},{long}",
        "radius": 5000,
        "types" : "hotel",
        "key": vacation_api
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")
narrowed_city_df["Hotel Name"] = hotel_df
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,city_id,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed,Hotel Name
0,16,Salta,0,AR,1667515073,16,-24.7859,-65.4117,73.99,12.66,Salta
1,55,Sur,0,OM,1667515087,49,22.5667,59.5289,77.25,10.07,Sur
2,164,Rawson,0,AR,1667514984,25,-43.3002,-65.1023,73.00,24.81,Rawson
3,176,Resistencia,0,AR,1667515072,32,-27.4606,-58.9839,76.98,5.75,Resistencia
4,201,Mitsamiouli,0,KM,1667515141,78,-11.3847,43.2844,76.17,9.78,Mitsamiouli


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Long"]]

In [32]:
# format map
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

#add previous heat layer as well
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…